### Add GPS Metadata fields

This notebook is designed for use in ArcGIS Pro 2.5+.  See [this topic](https://pro.arcgis.com/en/pro-app/2.8/arcpy/get-started/pro-notebooks.htm) For information on how to use Jupyter notebooks in ArcGIS Pro.

- Use this script to update an existing line or polygon feature class to support GPS metadata
- This adds a series of fields, necessary coded value domains, and enables attachments

# Example usage
add_gnss_fields(r"C:\Users\doug-m\Documents\ArcGIS\Projects\SampleProject\MyGDB.gdb\servicelines")

In [1]:
import arcpy
import os

def get_geodatabase_path(input_layer):
    """
    Gets the parent geodatabase of the layer
    :param input_layer: (string) The feature class to get the parent database of
    :return: (string) The path to the geodatabase
    """
    workspace = os.path.dirname(arcpy.Describe(input_layer).catalogPath)
    if [any(ext) for ext in ('.gdb', '.mdb', '.sde') if ext in os.path.splitext(workspace)]:
        return workspace
    else:
        return os.path.dirname(workspace)

In [2]:
def check_and_create_domains(geodatabase):
    """
    Checks if the Fix Type domain already exists.
    If the domains do not exist, they are created
    :param geodatabase: (string) the path to the geodatabase to check
    :return:
    """
    domains = arcpy.da.ListDomains(geodatabase)
    domain_names = [domain.name for domain in domains]
    if 'ESRI_FIX_TYPE_DOMAIN' in domain_names:
        for domain in domains:
            if domain.name == 'ESRI_FIX_TYPE_DOMAIN':
                # check if cvs 0,1,2,4,5 are in the codedValues
                values = [cv for cv in domain.codedValues]
                if not set(set([0, 1, 2, 4, 5])).issubset(values):
                    return "ESRI_FIX_TYPE_DOMAIN is missing a coded value pair."
    else:
        # Add the domain and values
      
        arcpy.CreateDomain_management(in_workspace=geodatabase,
                                      domain_name="ESRI_FIX_TYPE_DOMAIN",
                                      domain_description="Fix Type",
                                      field_type="SHORT",
                                      domain_type="CODED",
                                      split_policy="DEFAULT",
                                      merge_policy="DEFAULT")

        arcpy.AddCodedValueToDomain_management(in_workspace=geodatabase,
                                               domain_name="ESRI_FIX_TYPE_DOMAIN",
                                               code="0",
                                               code_description="Fix not valid")
        arcpy.AddCodedValueToDomain_management(in_workspace=geodatabase,
                                               domain_name="ESRI_FIX_TYPE_DOMAIN",
                                               code="1",
                                               code_description="GPS")
        arcpy.AddCodedValueToDomain_management(in_workspace=geodatabase,
                                               domain_name="ESRI_FIX_TYPE_DOMAIN",
                                               code="2",
                                               code_description="Differential GPS")
        arcpy.AddCodedValueToDomain_management(in_workspace=geodatabase,
                                               domain_name="ESRI_FIX_TYPE_DOMAIN",
                                               code="4",
                                               code_description="RTK Fixed")
        arcpy.AddCodedValueToDomain_management(in_workspace=geodatabase,
                                               domain_name="ESRI_FIX_TYPE_DOMAIN",
                                               code="5",
                                               code_description="RTK Float")

In [3]:
def add_gnss_fields(feature_layer):
    """
    This adds specific fields required for GPS units to
        auto-populate in the Field Maps application
        This will report errors if:
            1) Any of the fields already exist
            2) The input layer is not a line or polygon layer
            3) The layer is not found
            4) The layer is a shapefile
    """


    try:
        if not arcpy.Exists(feature_layer):
            return "Feature class: {} not found!".format(feature_layer)
        if arcpy.Describe(feature_layer).shapeType != "Polyline" and arcpy.Describe(feature_layer).shapeType != "Polygon":
            return "Feature class: {} is not a line or polygon layer. Use AddGPSMetadataFields_management instead".format(feature_layer). 
        if arcpy.Describe(feature_layer).dataType == "ShapeFile":
            return "ShapeFiles are not supported."
        
        # Check the domains to see if they exist and are valid
        # will update if necessary
        geodatabase = get_geodatabase_path(feature_layer)
        check_and_create_domains(geodatabase)      
        
        # Enable Attachments
        arcpy.EnableAttachments_management(feature_layer)
        
        # Add GNSS metadata fields
        existingFields = [field.name for field in arcpy.ListFields(feature_layer)]

        if 'esrignss_avg_h_rms' not in existingFields:
            arcpy.AddField_management(feature_layer,
                                      'esrignss_avg_h_rms',
                                      field_type="DOUBLE",
                                      field_alias='Average horizontal accuracy (m)',
                                      field_is_nullable="NULLABLE"
                                      )

        if 'esrignss_avg_v_rms' not in existingFields:
            arcpy.AddField_management(feature_layer,
                                      'esrignss_avg_v_rms',
                                      field_type="DOUBLE",
                                      field_alias='Average vertical accuracy (m)',
                                      field_is_nullable="NULLABLE"
                                      )

        if 'esrignss_worst_h_rms' not in existingFields:
            arcpy.AddField_management(feature_layer,
                                      'esrignss_worst_h_rms',
                                      field_type="DOUBLE",
                                      field_alias='Worst horizontal accuracy (m)',
                                      field_is_nullable="NULLABLE"
                                      )

        if 'esrignss_worst_v_rms' not in existingFields:
            arcpy.AddField_management(feature_layer,
                                      'esrignss_worst_v_rms',
                                      field_type="DOUBLE",
                                      field_alias='Worst vertical accuracy (m)',
                                      field_is_nullable="NULLABLE"
                                      )

        if 'esrignss_worst_fixtype' not in existingFields:
            arcpy.AddField_management(feature_layer,
                                      'esrignss_worst_fixtype',
                                      field_type="SHORT",
                                      field_alias='Worst Fix Type',
                                      field_is_nullable="NULLABLE",
                                      field_domain="ESRI_FIX_TYPE_DOMAIN"
                                      )

        if 'esrignss_manual_locations' not in existingFields:
            arcpy.AddField_management(feature_layer,
                                      'esrignss_manual_locations',
                                      field_type="LONG",
                                      field_alias='Number of manual locations',
                                      field_is_nullable="NULLABLE"
                                      )
        
        # Update GNSS metadata fields with Domains
        domainFields = [field for field in arcpy.ListFields(feature_layer) if field.name == 'esrignss_worst_fixtype']
        
        for field in domainFields:
            if field.name == 'esrignss_worst_fixtype' and not field.domain:
                arcpy.AssignDomainToField_management(feature_layer, field, 'ESRI_FIX_TYPE_DOMAIN')
        #        continue
            
        return "Successfully updated GPS Metadata fields with domains."
        
    except Exception as e:
        return "Failed"

In [ ]:
# Example usage
add_gnss_fields(r"C:\Users\doug-m\Documents\ArcGIS\Projects\SampleProject\MyGDB.gdb\servicelines")